<a href="https://colab.research.google.com/github/rishubhkhurana/nlp/blob/main/attention/Model1_BreakData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libs

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import re,os,sys,gc
import json
import torchtext
from torchtext.data import Field, Example, BucketIterator
from torchtext import data
import spacy, random
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import time, math

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Download data

In [ ]:
!wget https://github.com/allenai/Break/raw/master/break_dataset/Break-dataset.zip

--2020-12-28 12:57:36--  https://github.com/allenai/Break/raw/master/break_dataset/Break-dataset.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/allenai/Break/master/break_dataset/Break-dataset.zip [following]
--2020-12-28 12:57:37--  https://raw.githubusercontent.com/allenai/Break/master/break_dataset/Break-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15971078 (15M) [application/zip]
Saving to: ‘Break-dataset.zip’

Break-dataset.zip   100%[===================>]  15.23M  38.7MB/s    in 0.4s    

2020-12-28 12:57:37 (38.7 MB/s) - ‘Break-dataset.zip’ saved [15971078/15971078]



In [ ]:
!unzip Break-dataset.zip

Archive:  Break-dataset.zip
   creating: Break-dataset/logical-forms/
  inflating: Break-dataset/logical-forms/dev.csv  
  inflating: Break-dataset/logical-forms/test.csv  
  inflating: Break-dataset/logical-forms/train.csv  
   creating: Break-dataset/QDMR/
  inflating: Break-dataset/QDMR/dev.csv  
  inflating: Break-dataset/QDMR/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/test.csv  
  inflating: Break-dataset/QDMR/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/train.csv  
  inflating: Break-dataset/QDMR/train_lexicon_tokens.json  
   creating: Break-dataset/QDMR-high-level/
  inflating: Break-dataset/QDMR-high-level/dev.csv  
  inflating: Break-dataset/QDMR-high-level/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/test.csv  
  inflating: Break-dataset/QDMR-high-level/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/train.csv  
  inflating: Break-dataset/QDMR-high-level/train_lexicon_tokens.json  


## Creating Data

In [ ]:
train_df = pd.read_csv('Break-dataset/QDMR/train.csv')
valid_df = pd.read_csv('Break-dataset/QDMR/dev.csv')

In [ ]:
nlp = spacy.load('en')

In [ ]:
def tokenize_en(text):
    return  [tok.text for tok in nlp.tokenizer(text)]

In [ ]:
SRC = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>', batch_first=False)
TRG = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>', batch_first=False)

In [ ]:
fields = [('src',SRC),('trg',TRG)]

In [ ]:
train_examples = [Example.fromlist([s,t],fields) for s,t in zip(train_df['question_text'],train_df['decomposition'])]
valid_examples = [Example.fromlist([s,t],fields) for s,t in zip(valid_df['question_text'],valid_df['decomposition'])]

In [ ]:
train_ds = data.Dataset(train_examples, fields)
valid_ds = data.Dataset(train_examples, fields)

In [ ]:
SRC.build_vocab(train_ds, min_freq=2)
TRG.build_vocab(train_ds, min_freq=2)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 128
train_iterator, valid_iterator = BucketIterator.splits(
    (train_ds, valid_ds), 
    batch_size = BATCH_SIZE, 
    device = device,sort_key = lambda x: x.src)

## Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training Model

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9494, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(9499, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=9499, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,182,299 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 9m 32s
	Train Loss: 3.274 | Train PPL:  26.417
	 Val. Loss: 4.048 |  Val. PPL:  57.257
Epoch: 02 | Time: 9m 34s
	Train Loss: 2.419 | Train PPL:  11.234
	 Val. Loss: 4.088 |  Val. PPL:  59.638
Epoch: 03 | Time: 9m 31s
	Train Loss: 2.003 | Train PPL:   7.411
	 Val. Loss: 4.237 |  Val. PPL:  69.224
Epoch: 04 | Time: 9m 43s
	Train Loss: 1.807 | Train PPL:   6.092
	 Val. Loss: 4.099 |  Val. PPL:  60.257
Epoch: 05 | Time: 9m 37s
	Train Loss: 1.628 | Train PPL:   5.095
	 Val. Loss: 4.021 |  Val. PPL:  55.784
Epoch: 06 | Time: 9m 37s
	Train Loss: 1.500 | Train PPL:   4.480
	 Val. Loss: 4.052 |  Val. PPL:  57.514
Epoch: 07 | Time: 9m 37s
	Train Loss: 1.370 | Train PPL:   3.936
	 Val. Loss: 3.856 |  Val. PPL:  47.254
Epoch: 08 | Time: 9m 32s
	Train Loss: 1.259 | Train PPL:   3.521
	 Val. Loss: 3.780 |  Val. PPL:  43.812
Epoch: 09 | Time: 9m 37s
	Train Loss: 1.161 | Train PPL:   3.193
	 Val. Loss: 3.762 |  Val. PPL:  43.046
Epoch: 10 | Time: 9m 33s
	Train Loss: 1.121 | Train PPL